# Churn Analysis

In [1]:
# Import Packages
import pandas as pd
import numpy as np
np.random.seed(1)
import matplotlib.pyplot as plt

#import scikit-learn
#import jupyterlab
#import seaborn
#import xgboost

# Set Display Preferences
pd.set_option('display.max_columns', None)
pd.options.display.float_format = "{:.2f}".format

### Load & manipulate data

In [2]:
### UK Based ecom retailer sales from Jan 12, 2009 to Sep 12, 2011.
### Mainly sells unique all-occasion gift-ware.
df1 = pd.read_excel("data/online_retail_II.xlsx", sheet_name='Year 2009-2010', header=0)
df2 = pd.read_excel("data/online_retail_II.xlsx", sheet_name='Year 2010-2011', header=0)

### Note: Both excel sheets contain 2010 sales. Duplicates must be removed before concatenating.
# There is 1 week of overlap in the datasets
print(df1['InvoiceDate'].max())
print(df2['InvoiceDate'].min())

# Remove duplicate entries 
df1 = df1[~df1["Invoice"].isin(df2['Invoice'].unique())]
# Note: achives identical result as -> df1[df1['InvoiceDate']<df2['InvoiceDate'].min()] 

2010-12-09 20:01:00
2010-12-01 08:26:00


In [3]:
df = pd.concat([df1, df2], axis=0, ignore_index=True)
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom


### Define new factors

In [ ]:
### Sales are overwhelmingly from the UK.
uk_pct = len(df[df['Country']=='United Kingdom']) / len(df)
print(f'Percent of entries sold in the UK: {uk_pct*100:.2f}%')

### Create a binary variable indicating UK vs. Global.
df['UK_Ind'] = np.where(df['Country']=='United Kingdom', 1, 0)


Percent of entries sold in the UK: 91.88%


In [6]:
### Create a Invoice Amount column
df['Invoice Amount'] = df['Price'] * df['Quantity']

### Examining the Stock Codes, entries starting with alphabetic characters are atypical invoice entries that do not represent merchandise sales.
### Ex. Test products, adjustments, postage, carriage, etc.
df["StockCode"] = df["StockCode"].astype(str)

stockcode = (
    df.groupby(["StockCode", "Description"])["Quantity"]
      .sum()
      .reset_index()        
      .sort_values("StockCode", ascending=False)
    )

stockcode.head(25)

,StockCode,Description,Quantity
6569,m,Manual,5
6568,gift_0001_80,Dotcomgiftshop Gift Voucher £80.00,0
6567,gift_0001_70,Dotcomgiftshop Gift Voucher £70.00,1
6566,gift_0001_50,Dotcomgiftshop Gift Voucher £50.00,6
6565,gift_0001_40,Dotcomgiftshop Gift Voucher £40.00,5
6564,gift_0001_30,Dotcomgiftshop Gift Voucher £30.00,24
6563,gift_0001_20,to push order througha s stock was,10
6562,gift_0001_20,Dotcomgiftshop Gift Voucher £20.00,28
6561,gift_0001_10,Dotcomgiftshop Gift Voucher £10.00,15
6560,TEST002,This is a test product.,1


### Remove appropriate invoice entries

In [7]:
### Remove entries for Stock Codes beginning with alphabetic characters
df = df[df['StockCode'].str.match(r'^\d')]


### Null Customer IDs (required to determine churn). 
### Reason for missing Customer ID is unknown. For simplicity, these entries will be excluded from the analysis. This likely creates a source of bias.
### Recomendation is to update the ordering platform to require purchasers to create an account, generating a unique Customer ID.
null_id_pct = sum(df['Customer ID'].isnull()) / len(df)
null_id_pct
print(f'Percent of entries that are missing Customer ID: {null_id_pct*100:.2f}%')


### Some items have a price of $0. These entries will not count towards orders.
### Primarily consisting of car flags, stickers, shipping materials & invoice notes.
non_items = len(df[df['Price']==0]) / len(df)
print(f'Percent of entries that had a price of $0: {non_items*100:.2f}%')


### Remove entries is no Customer ID or Price = $0
df = df[~df['Customer ID'].isnull()]
df = df[df['Price']!=0]

Percent of entries that are missing Customer ID: 22.43%
Percent of entries that had a price of $0: 0.57%


### Aggregate data to Customer ID granualrity

In [8]:
df = df.reset_index()
df_aggregated = df.groupby(['Customer ID', 'UK_Ind'], as_index=False).agg(
    # First & last txn in the measurement period
    first_txn = ('InvoiceDate', 'min'),
    last_txn = ('InvoiceDate', 'max'),
    # Transaction Count
    total_txns = ('Invoice', 'nunique'),
    sale_txns = ('Invoice', lambda x: x[df.loc[x.index, 'Quantity'] >= 0].nunique()),
    return_txns = ('Invoice', lambda x: x[df.loc[x.index, 'Quantity'] < 0].nunique()),
    # Sales amount ($)
    total_amt = ('Invoice Amount', 'sum'),
    sale_amt = ('Invoice Amount', lambda x: x[df.loc[x.index, 'Quantity'] >= 0].sum()),
    return_amt = ('Invoice Amount', lambda x: x[df.loc[x.index, 'Quantity'] < 0].sum()),
    # Sale quantity
    total_qty = ('Quantity', 'sum'),
    sale_qty = ('Quantity', lambda x: x[df.loc[x.index, 'Quantity'] >= 0].sum()),
    return_qty = ('Quantity', lambda x: x[df.loc[x.index, 'Quantity'] < 0].sum()),
    # Variaty of SKUs
    total_skus = ('StockCode', 'nunique'),
    sale_skus = ('StockCode', lambda x: x[df.loc[x.index, 'Quantity'] >= 0].nunique())
    )

# Returns Percent
df_aggregated['return_pct'] = df_aggregated['return_amt'] / df_aggregated['sale_amt']
# Units per Transaction
df_aggregated['upt'] = df_aggregated['total_qty'] / df_aggregated['total_txns']
# Average Unit Retail
df_aggregated['aur'] = df_aggregated['total_amt'] / df_aggregated['total_qty']
# Average Order Value
df_aggregated['aov'] = df_aggregated['total_amt'] / df_aggregated['total_txns']


df_aggregated.head()

,Customer ID,UK_Ind,first_txn,last_txn,total_txns,sale_txns,return_txns,total_amt,sale_amt,return_amt,total_qty,sale_qty,return_qty,total_skus,sale_skus,return_pct,upt,aur,aov
0,12346.00,1,2010-03-02 13:08:00,2011-01-18 10:17:00,4,3,1,169.36,77352.96,-77183.60,24,74239,-74215,25,25,-1.00,6.00,7.06,42.34
1,12347.00,0,2010-10-31 14:20:00,2011-12-07 15:52:00,8,8,0,4921.53,4921.53,0.00,2967,2967,0,126,126,0.00,370.88,1.66,615.19
2,12348.00,0,2010-09-27 14:59:00,2011-09-25 13:13:00,5,5,0,1658.40,1658.40,0.00,2704,2704,0,24,24,0.00,540.80,0.61,331.68
3,12349.00,0,2009-12-04 12:49:00,2011-11-21 09:51:00,4,3,1,3654.54,3678.69,-24.15,1616,1621,-5,138,137,-0.01,404.00,2.26,913.63
4,12350.00,0,2011-02-02 16:01:00,2011-02-02 16:01:00,1,1,0,294.40,294.40,0.00,196,196,0,16,16,0.00,196.00,1.50,294.40


In [9]:
### Check to confirm there are no Customer IDs found in multiple markets/countries
cx_market = df_aggregated.groupby("Customer ID")["UK_Ind"].count()
multi_market_cx = sum(cx_market>1)
print('Number of customers in multiple markets:', multi_market_cx)

Number of customers in multiple markets: 0
